This link will open this notebook in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matonski/sentiment_classifier/blob/main/Sentiment_Classifier.ipynb)

Colab doesn't have the transformers package installed so this will install it.

In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 12.3MB/s 
     |████████████████████████████████| 890kB 67.8MB/s 
     |████████████████████████████████| 3.2MB 59.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=409bb618de600b91b8d55a20a7848fc15a49749ec74653bc24202ee6dfb52aff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from nltk.tree import Tree
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import requests, zipfile, io
r = requests.get( 'https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip' )
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Here is a link to the data. Unzip it into the same directory as this notebook.

https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip

The data is in a tree format with 5 categories of sentiment. The following function reads in the data and flattens each tree into a string. It also reduces the 5 categories to just 2 (postive and negative).

In [5]:
def sentiment_treebank_reader(filename):
    with open(filename, encoding='utf8') as f:
        X, y = [], []
        for line in f:
            tree = Tree.fromstring(line)
            label = int(tree.label())
            string = " ".join(tree.leaves())

            if label == 0 or label == 1: 
                y.append(0)
                X.append(string)
                
            elif label == 3 or label == 4:
                y.append(1)
                X.append(string)
    return X, y

In [6]:
X_str_dev, y_dev = sentiment_treebank_reader('trees/dev.txt')
X_str_train, y_train = sentiment_treebank_reader('trees/train.txt')
X_str_test, y_test = sentiment_treebank_reader('trees/test.txt')

We now define a dataset class in which the data consists of Bert-tokenized versions of the string data we just read in.

In [7]:
class SentimentDataset(Dataset):
    def __init__(self, strings, labels):
        self.strings = strings
        self.labels = labels
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        
    def __getitem__(self, index):
        string = self.strings[index]
        label = self.labels[index]
        
        encoding = self.tokenizer.encode_plus(string, 
                    add_special_tokens=True, return_attention_mask=True, padding='max_length')
        return (
                torch.tensor(encoding['input_ids']).to(device), 
                torch.tensor(encoding['attention_mask']).to(device), 
                torch.tensor(label, dtype=torch.long).to(device)
        )
    
    def __len__(self):
        return len(self.strings)

In [8]:
train_set = SentimentDataset(X_str_train, y_train)
dev_set = SentimentDataset(X_str_dev, y_dev)
test_set = SentimentDataset(X_str_test, y_test)

train_loader = DataLoader(train_set, batch_size=16)
dev_loader = DataLoader(dev_set, batch_size=16)
test_loader = DataLoader(test_set, batch_size=16)

As in the paper, the model is Bert with a classifier on the [CLS] output. We will then fine-tune the entire model on the training data.

In [9]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.classifier_layer = nn.Linear(768, 2)
    
    def forward(self, indices, mask):
        cls_output = self.bert(indices, attention_mask=mask)['pooler_output']
        return self.classifier_layer(cls_output)

In [10]:
model = SentimentClassifier()
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

This helper function evaluates the loss and accuracy on a given dataset.

In [11]:
def evaluate_model(model, data_loader, criterion, length):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        correct_predictions = 0
        for inputs, masks, labels in data_loader:
            outputs = model(inputs, masks)
            _, preds = torch.max(outputs, dim=1)
            loss = criterion(outputs, labels)
            correct_predictions += torch.sum(preds == labels)
            total_loss += loss.item()
    return total_loss, correct_predictions.item() / length

Here is the training loop.

In [ ]:
for epoch in range(2):
    print('Epoch', epoch + 1)
    model.train()

    for inputs, masks, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs, masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    train_loss, train_acc = evaluate_model(model, train_loader, criterion, len(train_set))
    dev_loss, dev_acc = evaluate_model(model, dev_loader, criterion, len(dev_set))
    print('Train loss', train_loss, '  accuracy', train_acc)
    print('Dev   loss', dev_loss, '  accuracy', dev_acc)

Epoch 1
Train loss 212.53419073671103   accuracy 0.773121387283237
Dev   loss 26.664875578135252   accuracy 0.7706422018348624
Epoch 2


In [ ]:
_, test_acc = evaluate_model(model, test_loader, criterion, len(test_set))
print('Test accuracy', test_acc)